In [1]:
from keras.engine import  Model
from keras.layers import Flatten, Dense, Input, Dropout
from keras.optimizers import Adam
from keras.preprocessing import image
from keras import backend as K
from keras_vggface.vggface import VGGFace
from keras_vggface import utils
import numpy as np
import os
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint
from keras import regularizers
%matplotlib inline

/home/gloria/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/gloria/tensorflow/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
print(K.tensorflow_backend._get_available_gpus())

['/job:localhost/replica:0/task:0/device:GPU:0']


In [3]:
K.clear_session()

In [4]:
vgg_base = VGGFace(include_top=False, input_shape=(100, 100, 3), pooling='max')

In [5]:
vgg_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 100, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 100, 100, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 100, 100, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 50, 50, 64)        0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 50, 50, 128)       73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 50, 50, 128)       147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 25, 25, 128)       0         
__________

In [6]:
# Add custom layers
last_layer = vgg_base.get_layer('global_max_pooling2d_1').output
X = Dropout(0.2)(last_layer)
# x = Flatten(name='flatten')(last_layer)
X = Dense(256, kernel_regularizer=regularizers.l2(0.0001), activation='relu', name='fc6')(X)
X = Dropout(0.2)(X)
X = Dense(128, kernel_regularizer=regularizers.l2(0.0001), activation='relu', name='fc7')(X)
X = Dropout(0.2)(X)
output = Dense(6, kernel_regularizer=regularizers.l2(0.0001), activation='softmax')(X)

In [7]:
# this is the model we will train
model = Model(inputs=vgg_base.input, outputs=output)

In [8]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in vgg_base.layers:
    layer.trainable = False

In [9]:
# Make sure weights are not trainable
model.trainable_weights

[<tf.Variable 'fc6/kernel:0' shape=(512, 256) dtype=float32_ref>,
 <tf.Variable 'fc6/bias:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'fc7/kernel:0' shape=(256, 128) dtype=float32_ref>,
 <tf.Variable 'fc7/bias:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'dense_1/kernel:0' shape=(128, 6) dtype=float32_ref>,
 <tf.Variable 'dense_1/bias:0' shape=(6,) dtype=float32_ref>]

In [10]:
model.compile(optimizer=Adam(lr=0.0001,clipnorm=1.0),loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 100, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 100, 100, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 100, 100, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 50, 50, 64)        0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 50, 50, 128)       73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 50, 50, 128)       147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 25, 25, 128)       0         
__________

In [14]:
BASE_DIR = '../project/all_females'

In [15]:
# IMPORTANT !
# LISTDIR returns the files as indexed by the filessystem
# not necessarily in alphanumeric order, so need to sort them!
N_IMAGES = len(os.listdir(BASE_DIR))
IMAGE_SIZE = (100, 100)

X_train = np.zeros((N_IMAGES, IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
y_train = np.zeros(N_IMAGES)

for idx, _im in enumerate(sorted(os.listdir(BASE_DIR))):
    # Change the image path with yours.
    _img = image.load_img(os.path.join(BASE_DIR, _im), target_size=IMAGE_SIZE)
    _x = image.img_to_array(_img)
    _x = np.expand_dims(_x, axis=0)
    X_train[idx, :, :, :] = utils.preprocess_input(_x, version=1) / .255 

In [16]:
# Make sure images are ok
assert ~np.any(np.isnan(X_train))

In [17]:
# Load ratings
ratings = np.genfromtxt('./Projekt_SGE_Assessment_ErikK.txt')
ratings = to_categorical(ratings, num_classes=6)

In [18]:
# Standardize ratings
ratings_z = (ratings - np.mean(ratings)) / np.std(ratings)

In [21]:
checkpointer = ModelCheckpoint(filepath='./weights.hdf5', verbose=1, save_best_only=True)
history = model.fit(X_train, ratings, batch_size=64, epochs=50, validation_split=0.2, callbacks=[checkpointer])

Train on 454 samples, validate on 114 samples
Epoch 1/50
454/454 [==============================] - 2s 4ms/step - loss: 10.2378 - acc: 0.3678 - val_loss: 11.6423 - val_acc: 0.2807

Epoch 00001: val_loss improved from inf to 11.64230, saving model to ./weights.hdf5
Epoch 2/50
454/454 [==============================] - 2s 4ms/step - loss: 10.2732 - acc: 0.3656 - val_loss: 11.6422 - val_acc: 0.2807

Epoch 00002: val_loss improved from 11.64230 to 11.64223, saving model to ./weights.hdf5
Epoch 3/50
454/454 [==============================] - 2s 4ms/step - loss: 10.3106 - acc: 0.3612 - val_loss: 11.6422 - val_acc: 0.2807

Epoch 00003: val_loss improved from 11.64223 to 11.64217, saving model to ./weights.hdf5
Epoch 4/50
454/454 [==============================] - 2s 4ms/step - loss: 10.3086 - acc: 0.3634 - val_loss: 11.6421 - val_acc: 0.2807

Epoch 00004: val_loss improved from 11.64217 to 11.64211, saving model to ./weights.hdf5
Epoch 5/50
454/454 [==============================] - 2s 4ms/st

454/454 [==============================] - 2s 4ms/step - loss: 10.3553 - acc: 0.3568 - val_loss: 11.6402 - val_acc: 0.2807

Epoch 00037: val_loss improved from 11.64023 to 11.64018, saving model to ./weights.hdf5
Epoch 38/50
454/454 [==============================] - 2s 4ms/step - loss: 10.3421 - acc: 0.3612 - val_loss: 11.6401 - val_acc: 0.2807

Epoch 00038: val_loss improved from 11.64018 to 11.64013, saving model to ./weights.hdf5
Epoch 39/50
454/454 [==============================] - 2s 4ms/step - loss: 10.3122 - acc: 0.3612 - val_loss: 11.6401 - val_acc: 0.2807

Epoch 00039: val_loss improved from 11.64013 to 11.64008, saving model to ./weights.hdf5
Epoch 40/50
454/454 [==============================] - 2s 4ms/step - loss: 10.2710 - acc: 0.3656 - val_loss: 11.6400 - val_acc: 0.2807

Epoch 00040: val_loss improved from 11.64008 to 11.64003, saving model to ./weights.hdf5
Epoch 41/50
454/454 [==============================] - 2s 4ms/step - loss: 10.2460 - acc: 0.3656 - val_loss: 11.

In [20]:
model.save('love_classifierv1.h5')